#### Name: {Broker}, {Carl}
#### Assignment: {6}

### Instructions
For each assignment use the provided template.  Use proper markdown and coding syntax.  

If you have note used Jupyter Notebooks before, please review: https://realpython.com/jupyter-notebook-introduction/

GIS Algorithms textbook

Chapter 3 Question 1.

# Chapter 3 Question 1.

We have mentioned that the Bentley–Ottmann algorithm is theoretically more efficient than the brute-force approach. Now it is time for us to explore this empirically. Write a Python program to randomly generate different numbers of line segments and use both approaches to test how they run.

In [33]:
#random ints
s = np.random.randint(0,8, (5, 2, 2))

In [34]:
s

array([[[1, 1],
        [2, 6]],

       [[5, 0],
        [1, 7]],

       [[7, 5],
        [3, 1]],

       [[7, 4],
        [2, 3]],

       [[3, 4],
        [6, 6]]])

## Brute-Force Approach

 (code from book)

In [1]:
import datetime
import numpy as np

In [2]:
#point.py

from math import sqrt
class Point():
    #A class for points in Cartesian coordinate systems.
    def __init__(self, x=None, y=None):
        self.x, self.y = x, y
    def __getitem__(self, i):
        if i==0: return self.x
        if i==1: return self.y
        return None
    def __len__(self):
        return 2
    def __eq__(self, other):
        if isinstance(other, Point):
            return self.x==other.x and self.y==other.y
        return NotImplemented
    def __ne__(self, other):
        result = self.__eq__(other)
        if result is NotImplemented:
            return result
        return not result
    def __lt__(self, other):
        if isinstance(other, Point):
            if self.x<other.x:
                return True
            elif self.x==other.x and self.y<other.y:
                return True
            return False
        return NotImplemented
    def __gt__(self, other):
        if isinstance(other, Point):
            if self.x>other.x:
                return True
            elif self.x==other.x and self.y>other.y:
                return True
            return False
        return NotImplemented
    def __ge__(self, other):
        if isinstance(other, Point):
            if self > other or self == other:
                return True
            else:
                return False
            return False
        return NotImplemented
    def __le__(self, other):
        if isinstance(other, Point):
            if self < other or self == other:
                return True
            else:
                return False
            return False
        return NotImplemented
    def __str__(self):
        if type(self.x) is int and type(self.y) is int:
            return "({0},{1})".format(self.x,self.y)
        else:
            return "({0:.1f}, {1:.1f})".format(self.x,self.y)
    def __repr__(self):
        if type(self.x) is int and type(self.y) is int:
            return "({0},{1})".format(self.x,self.y)
        else:
            return "({0:.1f}, {1:.1f})".format(self.x,self.y)
    def distance(self, other):
        return sqrt((self.x-other.x)**2 + (self.y-other.y)**2)

In [3]:
#sideplr.py

def sideplr(p, p1, p2):
    return int((p.x-p1.x)*(p2.y-p1.y)-(p2.x-p1.x)*(p.y-p1.y))

In [4]:
#linesegment.py

ENDPOINT = 0
INTERIOR = 1

class Segment:
    def __init__(self, e, p0, p1, c=None):
        if p0>=p1:
            p0,p1 = p1,p0 # p0 is always left
        self.edge = e # ID, in all edges
        self.lp = p0 # left point
        self.lp0 = p0 # original left point
        self.rp = p1 # right point
        self.status = ENDPOINT # status of segment
        self.c = c # c: feature ID
    def __eq__(self, other):
        if isinstance(other, Segment):
            return (self.lp==other.lp and self.rp==other.rp)\
                or (self.lp==other.rp and self.rp==other.lp)
        return NotImplemented
    def __ne__(self, other):
        result = self.__eq__(other)
        if result is NotImplemented:
            return result
        return not result
    def __lt__(self, other):
        if isinstance(other, Segment):
            if self.lp and other.lp:
                lr = sideplr(self.lp, other.lp, other.rp)
                if lr == 0:
                    lrr = sideplr(self.rp, other.lp, other.rp)
                    if other.lp.x < other.rp.x:
                        return lrr > 0
                    else:
                        return lrr < 0
                else:
                    if other.lp.x > other.rp.x:
                        return lr < 0
                    else:
                        return lr > 0
        return NotImplemented
    def __gt__(self, other):
        result = self.__lt__(other)
        if result is NotImplemented:
            return result
        return not result
    def __repr__(self):
        return "{0}".format(self.edge)
    def contains(self, p):
        if self.lp == p:
            return -1
        elif self.rp == p:
            return 1
        else:
            return 0

In [5]:
#intersection.py

def getIntersectionPoint(s1, s2):
    #Calculates the intersection point of two line segments
    #s1 and s2. This function assumes s1 and s2 intersect.
    #Intersection must be tested before calling this function.
    x1 = float(s1.lp0.x)
    y1 = float(s1.lp0.y)
    x2 = float(s1.rp.x)
    y2 = float(s1.rp.y)
    x3 = float(s2.lp0.x)
    y3 = float(s2.lp0.y)
    x4 = float(s2.rp.x)
    y4 = float(s2.rp.y)
    if s1.lp < s2.lp:
        x1,x2,y1,y2,x3,x4,y3,y4=x3,x4,y3,y4,x1,x2,y1,y2
    if x1 != x2:
        alpha1 = (y2-y1)/(x2-x1)
    if x3 != x4:
        alpha2 = (y4-y3)/(x4-x3)
    if x1 == x2: # s1 is vertical
        y = alpha2*(x1-x3)+y3
        return Point([x1, y])

    if x3==x4: # s2 is vertical
        y = alpha1*(x3-x1)+y1
        return Point([x3, y])
    if alpha1 == alpha2: # parallel lines
        return None
    # need to calculate
    x = (alpha1*x1-alpha2*x3+y3-y1)/(alpha1-alpha2)
    y = alpha1*(x-x1) + y1
    return Point(x, y)

def test_intersect(s1, s2):
    if s1==None or s2==None:
        return False
    # testing: s2 endpoints on the same side of s1
    lsign = sideplr(s2.lp0, s1.lp0, s1.rp)
    rsign = sideplr(s2.rp, s1.lp0, s1.rp)
    if lsign*rsign > 0:
        return False
    # testing: s1 endpoints on the same side of s2
    lsign = sideplr(s1.lp0, s2.lp0, s2.rp)
    rsign = sideplr(s1.rp, s2.lp0, s2.rp)
    if lsign*rsign > 0:
        return False
    return True

#test if lines intersect with brute force approach and calculate run time. 

In [45]:
#test if lines intersect with brute force approach

Start = datetime.datetime.now()

if __name__ == "__main__":
    p1 = Point(1, 1)
    p2 = Point(2, 6)
    p3 = Point(5, 0)
    p4 = Point(1, 7)
    p5 = Point(7, 5)
    p6 = Point(3, 1)
    p7 = Point(7, 4)
    p8 = Point(2, 3)
    p9 = Point(3, 4)
    p10 = Point(6, 6)
    
    s1 = Segment(0, p1, p2)
    s2 = Segment(1, p3, p4)
    s3 = Segment(2, p5, p6)
    s4 = Segment(3, p7, p8)
    s5 = Segment(4, p9, p10)
    
    if test_intersect(s1, s2):
        print(getIntersectionPoint(s1, s2))
    if test_intersect(s1, s3):
        print(getIntersectionPoint(s1, s3))
    if test_intersect(s1, s4):
        print(getIntersectionPoint(s1, s4))
    if test_intersect(s1, s5):
        print(getIntersectionPoint(s1, s5))
        
    if test_intersect(s2, s1):
        print(getIntersectionPoint(s2, s1))
    if test_intersect(s2, s3):
        print(getIntersectionPoint(s2, s3))
    if test_intersect(s2, s4):
        print(getIntersectionPoint(s2, s4))
    if test_intersect(s2, s5):
        print(getIntersectionPoint(s2, s5))
        
    if test_intersect(s3, s1):
        print(getIntersectionPoint(s3, s1))
    if test_intersect(s3, s2):
        print(getIntersectionPoint(s3, s2))
    if test_intersect(s3, s4):
        print(getIntersectionPoint(s3, s4))
    if test_intersect(s3, s5):
        print(getIntersectionPoint(s3, s5))

    if test_intersect(s4, s1):
        print(getIntersectionPoint(s4, s1))
    if test_intersect(s4, s2):
        print(getIntersectionPoint(s4, s2))
    if test_intersect(s4, s3):
        print(getIntersectionPoint(s4, s3))
    if test_intersect(s4, s5):
        print(getIntersectionPoint(s4, s5))
        
    if test_intersect(s5, s1):
        print(getIntersectionPoint(s5, s1))
    if test_intersect(s5, s2):
        print(getIntersectionPoint(s5, s2))
    if test_intersect(s5, s3):
        print(getIntersectionPoint(s5, s3))
    if test_intersect(s5, s4):
        print(getIntersectionPoint(s5, s4))
        
Finish = datetime.datetime.now()

print('This took', Finish - Start, 'seconds to complete.')


(1.9, 5.4)
(1.9, 5.4)
(3.9, 1.9)
(3.2, 3.2)
(3.9, 1.9)
(5.7, 3.7)
(3.2, 3.2)
(5.7, 3.7)
This took 0:00:00.000997 seconds to complete.


## Bentley–Ottmann Algorithm Approach

In [7]:
#line_seg_eventqueue.py

class Event:
    """
    An event in the sweep line algorithm. Each Event object
    stores the event point and the line segments associated with the
    point.
    """
    def __init__(self, p=None):
        self.edges = []     # line segments associated with the event
        self.p = p          # event point
    def __repr__(self):
        return "{0}{1}".format(self.p,self.edges)
        
class EventQueue:
    """
    An event queue in the sweep line algorithm. 
    """
    def __init__(self, lset):
        """
        Constructor of EventQueue.
        Input
          lset: a list of Segment objects. The left point of
                each segment is used to create an event
        Output
          A sorted list of events as a member of this class
        """
        if lset == None:
            return
        self.events = []
        for l in lset:
            e0 = Event(l.lp)
            inx = self.find(e0)
            if inx == -1:
                e0.edges.append(l)
                self.events.append(e0)
            else:
                self.events[inx].edges.append(l)
            e1 = Event(l.rp)
            if self.find(e1) == -1:
                self.events.append(e1)
        self.events.sort(key=lambda e: e.p)

    def add(self, e):
        """
        Adds event e to the queue, updates the list of events
        """
        self.events.append(e)
        self.events.sort(key=lambda e: e.p)

    def find(self, t):
        """
        Returns the index of event t if it is in the queue. 
        Otherwise, returns -1.
        """
        if isinstance(t, Event):
            p = t.p
        elif isinstance(t, Point):
            p = t
        else: return -1
        for e in self.events:
            if p == e.p:
                return self.events.index(e)
        return -1
        
    def is_empty(self):
        return len(self.events) == 0

In [8]:
#line_seg_intersection.py

import sys
sys.path.append("..")
from bintrees import AVLTree

#from .point import *
#from .intersection import *
#from .line_seg_eventqueue import *

def get_edges(t, p):
    """
    Gets the edges (segments) that contain point p as their right
    endpoint or in the interior
    """
    lr = []
    lc = []
    for s in AVLTree(t):
        if s.rp == p:
            lr.append(s)
        elif s.lp == p and s.status == INTERIOR:
            lc.append(s)
        elif sideplr(p, s.lp, s.rp) == 0:
            lc.append(s)
    return lr, lc

def get_lr(T, s):
    """
    Returns the left and right neighbors (branches) of s in T.
    """
    try:
        sl = T.floor_key(s)
    except KeyError:
        sl = None
    try:
        sr = T.ceiling_key(s)
    except KeyError:
        sr = None
    return sl, sr

def get_lrmost(T, segs):
    """
    Finds the leftmost and rightmost segments of segs in T
    """
    l = []
    for s in list(T):
        if s in segs:
            l.append(s)
    if len(l) < 1:
        return None, None
    return l[0], l[-1]

def find_new_event(s1, s2, p, q):
    """
    Tests if s1 intersects s2 at a point that is not in the event queue.
    When a new intersection point is found, a new event will be created
    and added to the event queue.
    Input:
       s1: line segment
       s2: line segment
       p: the point of the current event
       q: event queue
    Output:
       True if a new point is found, False otherwise
    Change: the content in the queue (q) may change.
    """
    ip = intersectx(s1, s2)
    if ip is None:
        return False
    if q.find(ip) is not -1:
        return False
    if ip.x>p.x or (ip.x==p.x and ip.y >= p.y):
        e0 = Event()
        e0.p = ip
        e0.edges = [s1, s2]
        q.add(e0)
    return True

def intersectx(s1, s2):
    """
    Tests intersection of 2 input segments. If intersection is possible,
    the actual intersection point will be calculated and returned.
    """
    if not test_intersect(s1, s2):
        return None
    p = getIntersectionPoint(s1, s2)   # an intersection
    return p

def intersections(psegs):
    """
    Implementation of the Bentley-Ottmann algorithm.
    Input
      psegs: a list of segments
    Output
      intpoints: a list of intersection points
    """
    eq = EventQueue(psegs)
    intpoints = []
    T = AVLTree()
    L=[]
    while not eq.is_empty():               # for all events
        e = eq.events.pop(0)               # remove the event
        p = e.p                            # get event point
        L = e.edges                        # segments with p as left end
        R,C = get_edges(T, p)              # p: right (R) and interior (C)
        if len(L+R+C) > 1:                 # Intersection at p among L+R+C
            for s in L+R+C:
                if not s.contains(p):      # if p is interior
                    s.lp = p               # change lp and
                    s.status = INTERIOR    # status
            intpoints.append(p)
            R,C = get_edges(T, p)
        for s in R+C:
            T.discard(s)
        for s in L+C:
            T.insert(s, str(s))
        if len(L+C) == 0:
            s = R[0]
            if s is not None:
                sl, sr = get_lr(T, s)
                find_new_event(sl, sr, p, eq)
        else:
            sp, spp = get_lrmost(T, L+C)
            try:
                sl = T.prev_key(sp)
            except KeyError:               # only on last key
                sl = None
            try:
                sr = T.succ_key(spp)
            except KeyError:               # only on last key
                sr = None
            find_new_event(sl, sp, p, eq)
            find_new_event(sr, spp, p, eq)
    return intpoints

#test if lines intersect with Bentley-Ottmann Algorithm approach and calculate run time.

In [46]:
psegs = [Segment(i, Point(s[i][0][0], s[i][0][1]),
                 Point(s[i][1][0], s[i][1][1]))
         for i in range(len(s))]

Start = datetime.datetime.now()
ints = intersections(psegs)
Finish = datetime.datetime.now()

print("There are", len(ints), "intersection points:")
print(ints)
print('This took', Finish - Start, 'seconds to complete.')

There are 5 intersection points:
[(1.9, 5.4), (2.0, 3.0), (3.2, 3.2), (3.9, 1.9), (5.7, 3.7)]
This took 0:00:00.002992 seconds to complete.


Observations:

In smaller numbers of line segments, the brute force method appears to be faster. With consideration to Big-O notation, as the number of line segments increase, so would the run time for the brute force method, more so than with the Bentley-Ottmann Algorithm. So the Bentley-Ottman Algorithm is far more scalable than the brute-force method. Which algorithm I would use would depend on usecase. 

I also noticed that when using the np random number generator, there were times when the Bentley-Ottmann Algorithm would get hung up and take a very long time to complete. As found on the discussion form, the culprit could be the line segments themselves (generating odd line segments). In a real world scenario, consideration to this problem would need to be included in the script to possibly exclude 'odd' line segments.  

End of notebook. -CJB 10/13/19